In [2]:
import psycopg2
import pandas as pd

In [3]:
# Parámetros de conexión
host = "localhost"
port = "5432"
dbname = "EcommerceDB"
user = "postgres"
password = "postgres123"

# Crear conexión
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    print(f"Conexión exitosa a la base de datos {dbname}")
except Exception as e:
    print(f"Error al conectar a la base de datos {dbname}:", e)


Conexión exitosa a la base de datos EcommerceDB


In [4]:
# VARIACION DE PRECIOS PARA PRODUCTOS EN ORDENES

query = """
SELECT 
    d.productoid,
    p.precio AS precio_oficial,
    d.preciounitario
FROM 
    public.detalleordenes d
JOIN 
    public.productos p ON d.productoid = p.productoid
"""
df = pd.read_sql_query(query, conn)

df["diferencia_pct"] = ((df["preciounitario"] - df["precio_oficial"]) / df["precio_oficial"]) * 100

resumen = df.groupby("productoid")["diferencia_pct"].agg(["min", "max"]).reset_index()
resumen.rename(columns={"min": "min_diferencia_pct", "max": "max_diferencia_pct"}, inplace=True)

print(resumen.head())

   productoid  min_diferencia_pct  max_diferencia_pct
0           1          -97.114203           42.781222
1           2          -98.221527          -37.449312
2           3          -85.683514          283.898761
3           4          -28.352235         3209.939960
4           5          -67.691923         1150.062516


C:\Users\gabriel.b.alvarez\AppData\Local\Temp\ipykernel_9652\2238761405.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [10]:
# EMAILS CON FORMATO INVALIDO

query_mal_formato = """ SELECT count(*)	FROM public.usuarios where email like '% %' """
query_totalidad = """ SELECT count(*) FROM public.usuarios """

cursor = conn.cursor()

cursor.execute(query_mal_formato)
resultado_mal_formato = cursor.fetchone()[0]
cursor.execute(query_totalidad)
resultado_totalidad = cursor.fetchone()[0]

porc_malos_sobre_total = (resultado_mal_formato/resultado_totalidad)*100

print(f'{porc_malos_sobre_total}%')

5.4%


In [14]:
conn.close()